In [1]:
# import必要套件
import pandas as pd
import numpy as np
import re

# 取出需要的dataset
df4 = pd.read_json('tt1424474-tt0006950.json')
df3 =pd.read_json('tt0010401-tt0144534.json')

# 合併兩個檔案
df = pd.concat([df3, df4], axis=0)

# 檢查有無重複值
df.index[df.index.duplicated()]

Index([], dtype='object')

In [2]:
# 觀察df發現有預算的欄位通常後面帶有(estimated)字樣,故
# 預算 只挑$ 後有(estimated) 並取最大值
df1 = df.applymap(lambda x: re.sub(r'[^\d]', '', x.replace('(estimated)', '')) if isinstance(x, str) and '(estimated)' in x and x.startswith('$') else np.nan)
merged_budget = pd.concat([df1[col] for col in df1.columns], axis=0)
merged_budget.dropna(inplace=True)
merged_budget = merged_budget.groupby(merged_budget.index).max()

In [3]:
# 觀查df發現有票房的欄位有很多個,並且都沒有(estimated)字樣
# 並且有多個金額,分別可能代表domestic, international, global, first week等(但資料不齊,無法斷定哪個類別一定會有)
# 收入 只挑$ 後無(estimated)取最大值
df2 = df.applymap(lambda x: re.sub(r'[^\d]', '', x) if isinstance(x, str) and x.startswith('$') and '(estimated)' not in x else np.nan)
merged_revenue = pd.concat([df2[col] for col in df2.columns], axis=0)
merged_revenue.dropna(inplace=True)
merged_revenue = merged_revenue.groupby(merged_revenue.index).max()

In [4]:
# 合併預算及收入
imdb_table = pd.concat([merged_budget, merged_revenue], axis=1)
# 更名
imdb_table = imdb_table.rename(columns={0: 'merged_budget', 1: 'merged_revenue'})
# 列出全部都不為NA的資料
imdb_table[imdb_table.notna().all(axis=1)]

,merged_budget,merged_revenue
tt0012190,800000,9183673
tt0014624,351000,11233
tt0018455,200000,121107
tt0018528,217000,4841
tt0018773,900000,27425
...,...,...
tt9686708,35000000,2180452
tt9779516,12000000,9868521
tt9857522,1000000,25618
tt9893250,14000000,1351662


In [5]:
#載入80萬筆的raw data
raw_df = pd.read_json('rawMovieWithTheNumAndKaggle.json')

In [6]:
# 確認資料筆數
len(imdb_table)

24283

In [7]:
# 確認資料筆數
len(raw_df)

860248

In [24]:
#將raw_df中imdb_id與imdb_table的index一樣的merged，標示變數名字為merged_df 
merged_df = pd.merge(raw_df, imdb_table, how='inner', left_on='imdb_id', right_index=True)
#將budget為零或nan的以merged_budget欄位填入 
merged_df['budget'] = merged_df.apply(lambda x: x['merged_budget'] if pd.isna(x['budget']) or x['budget'] == 0 else x['budget'], axis=1)
#將revenue為零或nan的以merged_revenue欄位填入 
merged_df['revenue'] = merged_df.apply(lambda x: x['merged_revenue'] if pd.isna(x['revenue']) or x['revenue'] == 0 else x['revenue'], axis=1)

In [25]:
# 可查看 merged_df
# merged_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline,merged_budget,merged_revenue
41,1094962,tt3498690,Sorority of the Damned,,1200000,NaN,en,,0.000,0,...,[],False,A reluctant handy man is pitted against evil w...,0.600,[United States of America],0,Post Production,,1200000,NaN
43,1094960,tt9465024,Camp Terror,,20000,NaN,en,,0.000,0,...,[Showcase Productions],False,The Quiet town of Forrest Lakes was paralyzed ...,0.600,[United States of America],110,Released,,20000,NaN
218,1094770,tt15249688,The Squad: Rise of the Chicano Squad,,3300000,NaN,en,2022-06-30,0.000,0,...,[],False,In 1979 Houston TX. was a boom town fantasy ci...,0.600,[United States of America],0,Released,,3300000,NaN
278,1094701,tt27002895,A Corpse for Christmas,,6000,NaN,en,2023-07-17,0.000,0,...,[Blood Sick Productions],False,NecroPhiladelphia splatter punks celebrate Chr...,1.442,[United States of America],0,In Production,"A sexy, sadistic, metal fest shot on VHS!",6000,NaN
544,1094405,tt3302070,Mondo Sacramento 2,,10000,NaN,en,2013-10-10,0.000,0,...,[Desperate Visions Productions],False,Mondo Sacramento 2 features 6 true tales from ...,0.828,[United States of America],112,Released,Mondo Sacramento 2- All Crazy and all based on...,10000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860238,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,"[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play.",50000000,6496125
860241,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,"[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.,NaN,556354
860242,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...,NaN,49033349
860243,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.,2300000,269435


In [26]:
# merged_df中有哪些revenue欄位為NA
merged_df['revenue'][merged_df['revenue'].isna()]

41        NaN
43        NaN
218       NaN
278       NaN
544       NaN
         ... 
859383    NaN
859396    NaN
859397    NaN
859402    NaN
859460    NaN
Name: revenue, Length: 15271, dtype: object

In [27]:
# merged_df中revenue為空值的index是哪些
merged_df['revenue'][merged_df['revenue'].isna()].index

Int64Index([    41,     43,    218,    278,    544,    590,    594,    599,
               857,   1073,
            ...
            859119, 859123, 859214, 859259, 859272, 859383, 859396, 859397,
            859402, 859460],
           dtype='int64', length=15271)

In [28]:
# 同上之理,將這些NA欄位補0
for index in merged_df['revenue'][merged_df['revenue'].isna()].index:
    merged_df.loc[index,'revenue'] = 0

for index in merged_df['budget'][merged_df['budget'].isna()].index:
    merged_df.loc[index,'budget'] = 0

In [29]:
# 檢查是否正確
merged_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline,merged_budget,merged_revenue
41,1094962,tt3498690,Sorority of the Damned,,1200000,0,en,,0.000,0,...,[],False,A reluctant handy man is pitted against evil w...,0.600,[United States of America],0,Post Production,,1200000,NaN
43,1094960,tt9465024,Camp Terror,,20000,0,en,,0.000,0,...,[Showcase Productions],False,The Quiet town of Forrest Lakes was paralyzed ...,0.600,[United States of America],110,Released,,20000,NaN
218,1094770,tt15249688,The Squad: Rise of the Chicano Squad,,3300000,0,en,2022-06-30,0.000,0,...,[],False,In 1979 Houston TX. was a boom town fantasy ci...,0.600,[United States of America],0,Released,,3300000,NaN
278,1094701,tt27002895,A Corpse for Christmas,,6000,0,en,2023-07-17,0.000,0,...,[Blood Sick Productions],False,NecroPhiladelphia splatter punks celebrate Chr...,1.442,[United States of America],0,In Production,"A sexy, sadistic, metal fest shot on VHS!",6000,NaN
544,1094405,tt3302070,Mondo Sacramento 2,,10000,0,en,2013-10-10,0.000,0,...,[Desperate Visions Productions],False,Mondo Sacramento 2 features 6 true tales from ...,0.828,[United States of America],112,Released,Mondo Sacramento 2- All Crazy and all based on...,10000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860238,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,"[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play.",50000000,6496125
860241,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,"[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.,NaN,556354
860242,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...,NaN,49033349
860243,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.,2300000,269435


In [30]:
# 檢查正確後,重新設定index
filtered_df = merged_df.query('budget != 0 and revenue != 0').reset_index(drop=True)

In [33]:
# 將不要的欄位去掉
filtered_df = filtered_df.drop(['merged_budget', 'merged_revenue'], axis=1)

In [35]:
# 抓出raw_df裡的預算票房都有的7620筆資料存成clean_df
clean_df = raw_df.loc[(raw_df['budget'] != 0) & (raw_df['budget'].notnull()) & 
                      (raw_df['revenue'] != 0) & (raw_df['revenue'].notnull()) &
                      (raw_df['production_countries'].apply(lambda x: 'United States of America' in x))]

In [37]:
#將TMDB的7620與IMDB的2652筆合併
fin_imdb_merge_df = pd.concat([clean_df, filtered_df], ignore_index=True)

In [38]:
# 確認有幾筆資料
len(fin_imdb_merge_df)

10272

In [39]:
# 刪除movie_id重複的資料
fin_imdb_merge_df.drop_duplicates(['movie_id'], inplace=True)

In [40]:
# 確認還有幾筆資料
len(fin_imdb_merge_df)

9593